In [1]:
%cd /content
!rm -rf *
!pip3 install mpi4py
!pip3 install ipyparallel

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp38-cp38-linux_x86_64.whl size=5997370 sha256=ab519e6f3656bdb636ce740cf1166a0bcbb817f0a409818e5f8d1e9806c143d0
  Stored in directory: /root/.cache/pip/wheels/f3/35/48/0b9a7076995eea5ea64a7e4bc3f0f342f453080795276264e7
Successfully built mpi4py
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.1 MB/s eta 0:00:00


In [2]:
import os
results = r'/content/results'

list_names = ['SGD','Adam','Adagrad','RMSprop','Adadelta']
if not os.path.exists(results):
    os.makedirs(results)
for name in list_names:
  optim_dir = f'{results}/{name}'
  if not os.path.exists(optim_dir):
    os.makedirs(optim_dir)

In [31]:
import torch
import torchvision

In [32]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
random_seed = 1

In [33]:
# train_loader,test_loader = make_train_loader_test_loader()

In [34]:
# examples = enumerate(test_loader)
# batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape

In [35]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD, Adam, Adagrad,RMSprop,Adadelta

In [24]:
def s_to_bitlist(s):
      ords = (ord(c) for c in s)
      shifts = (7, 6, 5, 4, 3, 2, 1, 0)
      return [(o >> shift) & 1 for o in ords for shift in shifts]
def bitlist_to_chars(bl):
  bi = iter(bl)
  bytes = zip(*(bi,) * 8)
  shifts = (7, 6, 5, 4, 3, 2, 1, 0)
  for byte in bytes:
    yield chr(sum(bit << s for bit, s in zip(byte, shifts)))

def bitlist_to_s(bl):
  return ''.join(bitlist_to_chars(bl))

a = s_to_bitlist('Hi')

bitlist_to_s(a)

'Hi'

###Training parallel

In [ ]:
import numpy as np
my_str = "hello"
my_str_as_bytes = np.array(bytes(my_str,'UTF-8'))
print(type(my_str_as_bytes)) # ensure it is byte representation
my_decoded_str = my_str_as_bytes.decode()
print(type(my_decoded_str)) # ensure it is string representation

In [90]:
# %localhost slots=4
! mpirun -n 4 --allow-run-as-root --oversubscribe python3  mpiTrain.py

Optimizer: Adam - Start
Optimizer: Adagrad - Start
Optimizer: SGD - Start

Test set: Avg. loss: 2.3044, Accuracy: 986/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.287512

Test set: Avg. loss: 2.2976, Accuracy: 906/10000 (9%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.321400

Test set: Avg. loss: 2.3218, Accuracy: 622/10000 (6%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299502
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.300504
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.249135
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.273666
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.113180
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.235998
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.329293
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.955084
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.242866
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.672861
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.355936
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.190251
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.217985
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.197073
T

And that's it. With just 3 epochs of training we already managed to achieve 97% accuracy on the test set! We started out with randomly initialized parameters and as expected only got about 10% accuracy on the test set before starting the training.

Let's plot our training curve.

In [92]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as offline

import numpy as np

titles_names = [f'Plot {optimizer}' for optimizer in list_names]
# len(list_names), 1
fig = make_subplots(
    rows=len(list_names), cols=1,
    subplot_titles=titles_names)

for i in range(4-1):
    name = list_names[i]
    with open(f'/content/results/{name}/train_losses.txt', 'r') as file:
        train_losses = list(map(float,file.read().replace('\n', '').replace('[','').replace(']','').split(sep=', ')))
    
    with open(f'/content/results/{name}/train_counter.txt', 'r') as file:
        train_counter = list(map(float,file.read().replace('\n', '').replace('[','').replace(']','').split(sep=', ')))

    with open(f'/content/results/{name}/test_losses.txt', 'r') as file:
        test_losses = list(map(float,file.read().replace('\n', '').replace('[','').replace(']','').split(sep=', ')))
    
    with open(f'/content/results/{name}/test_counter.txt', 'r') as file:
        test_counter = list(map(float,file.read().replace('\n', '').replace('[','').replace(']','').split(sep=', '))) 

    
    fig.add_trace(go.Scatter(x=train_counter,y=train_losses),               
                  row=i+1, col=1)
    
    fig.add_scatter(
        x=test_counter,
        y=test_losses, 
        mode="markers",                
        marker=dict(size=20,color="LightSeaGreen"),                
        #name="a", 
        row=i+1, col=1)

fig.update_layout(height=500, width=700,
                  title_text="Optimizers")

fig.show()